In [ ]:
#installing streamlit
!pip install -q streamlit
#installing easyocr library
!pip install easyocr

In [12]:
%%writefile app.py
#importing dependcies
import streamlit as st
import easyocr as ocr
from PIL import Image
import pandas as pd
import numpy as np
import re
import sqlite3

#Title for webapp
st.title("BizCardX: Extracting Business Card")

#File uploader-Image uploading
img = st.file_uploader(label="UPLOAD YOUR BUISNESS CARD",type=['png','jpg','jpeg'])
#Reading the image 
reader = ocr.Reader(['en'])

if img is not None:
    ex_image = Image.open(img)
    result = reader.readtext(np.array(ex_image))
    #creating a empty list to store extracted details 
    l = []
    for res in result:
       text = res[1]
       l.append(text)
       
       
    #dictionary for creating dataframe
    fn_dict = {'Company':[],'Co-ordinator':[],'Designation':[],'Phone_Number':[],'Email_Address':[],'Website':[],'State':[],'Pin_Code':[]}
    
    
    #extracting the name and Designation
    fn_dict['Co-ordinator'].append(l[0])
    fn_dict['Designation'].append(l[1])
    
    #Extracting the phone number
    ph_pattern = r'^\+?\d{1,3}-\d{3}-\d{4}$'
    for item in l:
      match = re.search(ph_pattern, item)      
      if match:
        phone_number = match.group()
        fn_dict['Phone_Number'].append(phone_number)
 
    #Extracting the Email_id
    mail_pattern = r'^h.*'
    for item in l:
      match = re.search(mail_pattern, item) 
      if match:
        mail = match.group()
        fn_dict['Email_Address'].append(mail)
    
    #Extracting the Website of comapny
    web_pattern = r'^[wW].*'
    for item in l:
      match = re.search(web_pattern, item) 
      if match:
        web = match.group()
        fn_dict['Website'].append(web)

    #Extracting the location-State
    state_pattern = r'\bTamilNadu\b'
    for item in l:
      match = re.search(state_pattern, item) 
      if match:
        state = match.group(0)
        fn_dict['State'].append(state)

    #Extracting the location-Pincode
    pin_pattern = r'\b\d{7}\b'
    for item in l:
      match = re.search(pin_pattern, item) 
      if match:
        pin = match.group(0)
        fn_dict['Pin_Code'].append(pin)
   
    #Extracting the Comapny name
    comapny_pattern = r'\b(BORCELLE)\b'
    for item in l:
      match = re.search(comapny_pattern, item) 
      if match:
        company = match.group(0)        
        fn_dict['Company'].append(company)       
          
    #creating the dataframe
    df = pd.DataFrame(fn_dict)
    df_view = df.T
    
    #viewing the DataFrame
    st.dataframe(df_view)
    
    #uploading to sql
    if st.button('Upload to SQL'):
      conn = sqlite3.connect('Buisness.db')
      df.to_sql(name = "items",con = conn, if_exists = 'append')
      st.write('Uploaded Sucessfully')

Overwriting app.py


**Streamlit running**

*  Run all cells below and click the link generated




In [13]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.32s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [14]:
!streamlit run /content/app.py &>/content/logs.txt &

In [15]:
!npx localtunnel --port 8501

npx: installed 22 in 1.693s
your url is: https://tasty-doodles-look-35-247-89-84.loca.lt
^C
